In [34]:
import pandas as pd
import json
import os

folder_path = "Sample Data"
file_name = "customers.csv"
file_path = os.path.join(folder_path, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    df = pd.read_csv(f)
print(df)
# Convert column names dynamically
df.columns = (
    df.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()            # Convert to lowercase
    .str.replace(" ", "_")  # Replace spaces with underscores
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
)

    Customer ID       Customer Name Region                      Email  \
0             1         Lauren Barr  North        morganamy@gmail.com   
1             2          Mary Beard  South      shannon87@hotmail.com   
2             3     Keith Rodriguez  North   charlesbarnes@reeves.com   
3             4       Karen Frazier   East       fhernandez@yahoo.com   
4             5       James Harrell  South         john44@mcbride.biz   
..          ...                 ...    ...                        ...   
95           96     Christina Lopez  South    melindaoneill@yahoo.com   
96           97      Laura Mitchell   West          erica48@yahoo.com   
97           98  Christopher Chavez  South  stephaniecooper@gmail.com   
98           99     Victoria Curtis  South     davidgilbert@gmail.com   
99          100       Valerie Wells  North         angel71@rogers.org   

                     Phone     Industry    Customer City    Country    Zip  
0   001-508-070-2916x67243   Healthcare    Joh

In [36]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [42]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS task_1_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS task_1_mg")
        cs.execute("USE WAREHOUSE task_1_warehouse_mg")
        cs.execute("USE SCHEMA task_1_mg")
        runQueries(cs,df)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

In [44]:
def runQueries(cs,df):    
    df = df.astype(str).apply(lambda col: col.map(lambda x: None if pd.isna(x) or x == "" or str(x).strip().lower() == "nan" else x))

    # Generate CREATE TABLE statement dynamically
    table_name = "article_table_pandas_1"
    columns = ", ".join([f'{col} STRING' for col in df.columns])
    create_table_sql = f'CREATE OR REPLACE TABLE {table_name} ({columns})'
    cs.execute(create_table_sql)

    # Insert data into Snowflake
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        insert_sql = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({placeholders})'
        cs.execute(insert_sql, tuple(row))